In [3]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import time

In [5]:
!which chromedriver

/c/Users/benkr/bin/chromedriver


In [6]:
executable_path = {'executable_path': '/c/Users/benkr/bin/chromedriver'}
browser = Browser('chrome', **executable_path)
mars={}

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
soup = BeautifulSoup(browser.html,"html.parser")
results = soup.find_all("div", class_="list_text")
#result=result.find_all("div", class_="content_title")
title = []
paragraph = []

for result in results:
    title.append(result.find("div", class_="content_title").a.text)
    paragraph.append(result.find("div", class_="article_teaser_body").text)
#mars["news_title"]=title[0]   
#mars["news_paragraph"] = paragraph[0]
newstitle=title[0]
newsparagraph=paragraph[0]
mars["news_title"] = newstitle 
mars["news_paragraph"] = newsparagraph

In [5]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
browser.find_by_id("full_image").click()
browser.find_link_by_partial_text('more info').click()
soup = BeautifulSoup(browser.html,"html.parser")
results = soup.find_all("figure", class_="lede")
#results.a.img["src"]
link=results[0].a.img["src"]
featured_image_url = 'https://www.jpl.nasa.gov' + link
mars["featured_image"] = featured_image_url

C:\Users\benkr\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [6]:
mars

{'news_title': 'MOXIE Could Help Future Rockets Launch Off Mars',
 'news_paragraph': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.",
 'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA12826_hires.jpg'}

In [7]:
# Mars Facts
url_facts = "https://space-facts.com/mars/"
table = pd.read_html(url_facts)
table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [8]:
df_mars_facts = table[0]
df_mars_facts.columns = ["Parameter", "Values"]
df_mars_facts.set_index(["Parameter"])

,Values
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [9]:
mars_html_table = df_mars_facts.to_html()
mars_html_table = mars_html_table.replace("\n", "")
mars["facts"]=mars_html_table

In [10]:
# URL of USGS Astrogeology site to be scraped
# to obtain high resolution images for each of Mar's hemispheres
usgs_astrogeology_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_astrogeology_url)

# Initialize the list for the dictionary of hemisphere images 
hemisphere_image_urls = []

# Get HTML page with the browser; create BeautifulSoup object; parse with 'html.parser'
usgs_astrogeology_html = browser.html
all_hemisphere_data = BeautifulSoup(usgs_astrogeology_html, 'html.parser')
#all_hemisphere_data

In [11]:
hemisphere_results = all_hemisphere_data.find('div', class_='collapsible results').find_all('div',class_='item')
hemisphere_results

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [13]:
# Loop through hemisphere results
for each_hemisphere in hemisphere_results:
    
    # Get each hemisphere title 
    hem_title = each_hemisphere.find('div', class_='description').find('a', class_='itemLink product-item').h3.text
    # Exclude the word 'Enhanced'
    short_hem_title = ' '.join(hem_title.split()[0:-1])
     
    # Get each hemisphere image URL
    base_hem_url = 'https://astrogeology.usgs.gov'

    each_hem_image_url = base_hem_url + each_hemisphere.find('a',class_='itemLink product-item')['href']

    
    browser.visit(each_hem_image_url)
    time.sleep(2)
    each_hem_img_html = browser.html
    each_hem_data = BeautifulSoup(each_hem_img_html, 'html.parser')
    full_image_url = each_hem_data.find('div',class_='downloads').a['href']
    
    each_hemisphere_image = {
        "title" : short_hem_title,
        "image_url" : full_image_url
    }
    print(each_hemisphere_image)
    # Append each hemisphere info to the list of all hemipheres  
    hemisphere_image_urls.append(each_hemisphere_image)
mars["hemispheres"] = hemisphere_image_urls

{'title': 'Cerberus Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'title': 'Schiaparelli Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'title': 'Syrtis Major Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'title': 'Valles Marineris Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}


In [14]:
mars

{'news_title': 'MOXIE Could Help Future Rockets Launch Off Mars',
 'news_paragraph': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.",
 'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA12826_hires.jpg',
 'facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Parameter</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </t